In [125]:
import pandas as pd
df_1 = pd.read_csv('tabularData.csv')
df_1.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168


In [126]:
df_1.shape

(112120, 11)

In [127]:
null_values = df_1.isnull()

# Display the DataFrame with null values (True for null values)
print(df_1.isnull().sum())

Image Index                    0
Finding Labels                 0
Follow-up #                    0
Patient ID                     0
Patient Age                    0
Patient Gender                 0
View Position                  0
OriginalImage[Width            0
Height]                        0
OriginalImagePixelSpacing[x    0
y]                             0
dtype: int64


In [128]:
t = df_1['Finding Labels'].unique()

In [129]:
len(t)

836

In [130]:
df_1['View Position'].nunique()

2

In [131]:
df_encoded = pd.get_dummies(df_1, columns=['View Position', 'Patient Gender'], drop_first=True)

In [132]:
df_encoded


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],View Position_PA,Patient Gender_M
0,00000001_000.png,Cardiomegaly,0,1,57,2682,2749,0.143,0.143,True,True
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,2894,2729,0.143,0.143,True,True
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,2500,2048,0.168,0.168,True,True
3,00000002_000.png,No Finding,0,2,80,2500,2048,0.171,0.171,True,True
4,00000003_001.png,Hernia,0,3,74,2500,2048,0.168,0.168,True,False
...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,2048,2500,0.168,0.168,True,True
112116,00030802_000.png,No Finding,0,30802,28,2048,2500,0.168,0.168,True,True
112117,00030803_000.png,No Finding,0,30803,42,2048,2500,0.168,0.168,True,False
112118,00030804_000.png,No Finding,0,30804,29,2048,2500,0.168,0.168,True,False


In [133]:
df_encoded[['View Position_PA', 'Patient Gender_M']] = df_encoded[['View Position_PA', 'Patient Gender_M']].astype(int)

In [134]:
df_encoded

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],View Position_PA,Patient Gender_M
0,00000001_000.png,Cardiomegaly,0,1,57,2682,2749,0.143,0.143,1,1
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,2894,2729,0.143,0.143,1,1
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,2500,2048,0.168,0.168,1,1
3,00000002_000.png,No Finding,0,2,80,2500,2048,0.171,0.171,1,1
4,00000003_001.png,Hernia,0,3,74,2500,2048,0.168,0.168,1,0
...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,2048,2500,0.168,0.168,1,1
112116,00030802_000.png,No Finding,0,30802,28,2048,2500,0.168,0.168,1,1
112117,00030803_000.png,No Finding,0,30803,42,2048,2500,0.168,0.168,1,0
112118,00030804_000.png,No Finding,0,30804,29,2048,2500,0.168,0.168,1,0


In [135]:
from sklearn.preprocessing import MultiLabelBinarizer
# Extract and split labels, ensuring unique labels using set
labels = df_encoded['Finding Labels'].str.split('|').apply(lambda x: list(set(x)))

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Transform the labels into binary matrix (multi-hot encoded)
multi_hot_encoded = mlb.fit_transform(labels)

label_classes = list(mlb.classes_)

# Create a new column with vectors of encoded values
df_encoded['Encoded Vectors'] = [list(row) for row in multi_hot_encoded]

# If needed, concatenate this with the original DataFrame
final_df = df_encoded.copy()  # or pd.concat([df, df_encoded], axis=1) if original df is needed


In [136]:
final_df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],View Position_PA,Patient Gender_M,Encoded Vectors
0,00000001_000.png,Cardiomegaly,0,1,57,2682,2749,0.143,0.143,1,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,2894,2729,0.143,0.143,1,1,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,2500,2048,0.168,0.168,1,1,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,00000002_000.png,No Finding,0,2,80,2500,2048,0.171,0.171,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
4,00000003_001.png,Hernia,0,3,74,2500,2048,0.168,0.168,1,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,2048,2500,0.168,0.168,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]"
112116,00030802_000.png,No Finding,0,30802,28,2048,2500,0.168,0.168,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
112117,00030803_000.png,No Finding,0,30803,42,2048,2500,0.168,0.168,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
112118,00030804_000.png,No Finding,0,30804,29,2048,2500,0.168,0.168,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"


In [137]:
len(final_df['Encoded Vectors'][0])

15

In [138]:
labels_2 = df_encoded['Finding Labels'].str.split('|').tolist()

In [139]:
flat_labels = [item for sublist in labels_2 for item in sublist]

# Get unique labels using a set
unique_labels = set(flat_labels)

# Convert the set back to a list if you want to work with it as a list
unique_labels = list(unique_labels)

# Print unique labels
print(len(unique_labels))

15


In [140]:
df_2 = pd.read_csv('image_vectors.csv')

In [141]:
df_2.head(10)

,image_name,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_2038,feature_2039,feature_2040,feature_2041,feature_2042,feature_2043,feature_2044,feature_2045,feature_2046,feature_2047
0,00018805_001.png,0.418487,0.306601,0.801291,0.044648,0.607149,0.503188,0.556803,0.409647,0.551391,...,0.204681,0.078678,0.227979,0.145360,0.125559,0.100496,0.033357,0.070022,0.511545,0.360500
1,00005868_002.png,0.373129,0.462354,1.194659,0.144330,0.310254,1.676204,0.455906,0.273282,0.828891,...,0.308831,0.009194,0.044123,0.297138,0.610502,0.094704,0.191540,0.219701,1.179693,0.648014
2,00009663_000.png,0.050278,0.744042,1.508330,0.488489,0.378242,0.385282,0.224698,0.122177,0.159800,...,0.169191,0.000791,0.007508,0.506602,0.230475,0.250326,0.054024,0.078933,0.257028,0.110671
3,00022156_007.png,0.066088,0.777098,1.181380,0.404282,0.221584,0.366294,0.410837,0.117063,0.229142,...,0.247339,0.027537,0.042472,0.147641,0.250765,0.095297,0.134013,0.236017,0.210253,1.181115
4,00005395_004.png,0.041102,0.181218,0.742273,0.150364,0.119664,0.148466,0.248920,0.109956,0.305178,...,0.144721,0.013806,0.025944,0.141631,0.354130,0.111998,0.050722,0.037256,0.512696,0.782948
5,00006966_004.png,0.159714,1.358857,1.306791,0.197567,0.075864,0.826678,0.572982,0.374754,0.360177,...,0.372673,0.117993,0.094834,0.351271,0.822779,0.061681,0.059519,0.708100,1.156346,0.380706
6,00000324_013.png,0.191458,0.545136,1.048456,0.539831,0.424234,0.826224,0.434510,0.240744,0.083549,...,0.064091,0.127144,0.048714,0.288780,0.361534,0.051720,0.000000,0.520523,0.160173,0.121537
7,00007442_018.png,0.860041,0.454337,1.521246,0.101389,0.243245,0.624360,0.323650,0.034314,0.413795,...,0.082985,0.050778,0.006051,0.128206,0.287190,0.150563,0.095725,0.038282,0.472744,1.630677
8,00003923_012.png,0.097573,0.273693,0.968594,0.291712,0.731622,0.471571,0.241238,0.046700,0.555825,...,0.097095,0.003563,0.002323,0.247713,0.333208,0.034942,0.195362,0.038518,0.144311,0.924252
9,00028483_007.png,0.059173,0.537611,0.908132,0.142843,0.121920,0.520985,0.370797,0.190313,0.184255,...,0.236933,0.188259,0.033841,0.305419,0.349624,0.017424,0.003448,0.223069,0.275798,0.376505


In [142]:
merged_df = pd.merge(final_df, df_2, left_on="Image Index", right_on="image_name", how="inner") 
merged_df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],View Position_PA,...,feature_2038,feature_2039,feature_2040,feature_2041,feature_2042,feature_2043,feature_2044,feature_2045,feature_2046,feature_2047
0,00000001_000.png,Cardiomegaly,0,1,57,2682,2749,0.143,0.143,1,...,0.054727,0.000000,0.074869,0.182716,0.414462,0.046612,0.175924,0.550939,0.302225,0.506780
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,2894,2729,0.143,0.143,1,...,0.089889,0.017977,0.091583,0.169574,0.507109,0.043761,0.138473,0.652440,0.527583,0.292152
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,2500,2048,0.168,0.168,1,...,0.111583,0.033380,0.097974,0.331221,0.068201,0.191576,0.070453,0.076101,0.193749,0.167181
3,00000002_000.png,No Finding,0,2,80,2500,2048,0.171,0.171,1,...,0.246135,0.008017,0.024985,0.318914,0.556515,0.178968,0.038388,0.102484,0.487945,0.355339
4,00000003_001.png,Hernia,0,3,74,2500,2048,0.168,0.168,1,...,0.079363,0.079900,0.128630,0.254023,0.143975,0.196335,0.038926,0.103639,0.886098,0.308224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,2048,2500,0.168,0.168,1,...,0.160762,0.039061,0.025498,0.181054,0.096583,0.152477,0.188412,0.016646,0.674780,1.414213
112116,00030802_000.png,No Finding,0,30802,28,2048,2500,0.168,0.168,1,...,0.168288,0.027486,0.026430,0.207038,0.226278,0.151056,0.076672,0.054659,1.460316,0.946452
112117,00030803_000.png,No Finding,0,30803,42,2048,2500,0.168,0.168,1,...,0.165465,0.026710,0.016028,0.338843,0.551825,0.187382,0.047657,0.027959,0.839741,0.855948
112118,00030804_000.png,No Finding,0,30804,29,2048,2500,0.168,0.168,1,...,0.162420,0.112969,0.012846,0.148053,0.434081,0.050531,0.029546,0.078464,0.918192,0.598955


In [143]:
merged_df['Encoded Vectors']

0         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1         [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2         [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
4         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
                              ...                      
112115    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
112116    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
112117    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
112118    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
112119    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Name: Encoded Vectors, Length: 112120, dtype: object

In [144]:
merged_df['image_name']

0         00000001_000.png
1         00000001_001.png
2         00000001_002.png
3         00000002_000.png
4         00000003_001.png
                ...       
112115    00030801_001.png
112116    00030802_000.png
112117    00030803_000.png
112118    00030804_000.png
112119    00030805_000.png
Name: image_name, Length: 112120, dtype: object

In [145]:
merged_df.drop(columns=["image_name", "Image Index", "Finding Labels"], inplace=True)

In [146]:
merged_df

,Follow-up #,Patient ID,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],View Position_PA,Patient Gender_M,Encoded Vectors,...,feature_2038,feature_2039,feature_2040,feature_2041,feature_2042,feature_2043,feature_2044,feature_2045,feature_2046,feature_2047
0,0,1,57,2682,2749,0.143,0.143,1,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",...,0.054727,0.000000,0.074869,0.182716,0.414462,0.046612,0.175924,0.550939,0.302225,0.506780
1,1,1,58,2894,2729,0.143,0.143,1,1,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",...,0.089889,0.017977,0.091583,0.169574,0.507109,0.043761,0.138473,0.652440,0.527583,0.292152
2,2,1,58,2500,2048,0.168,0.168,1,1,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",...,0.111583,0.033380,0.097974,0.331221,0.068201,0.191576,0.070453,0.076101,0.193749,0.167181
3,0,2,80,2500,2048,0.171,0.171,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",...,0.246135,0.008017,0.024985,0.318914,0.556515,0.178968,0.038388,0.102484,0.487945,0.355339
4,0,3,74,2500,2048,0.168,0.168,1,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",...,0.079363,0.079900,0.128630,0.254023,0.143975,0.196335,0.038926,0.103639,0.886098,0.308224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,1,30801,38,2048,2500,0.168,0.168,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]",...,0.160762,0.039061,0.025498,0.181054,0.096583,0.152477,0.188412,0.016646,0.674780,1.414213
112116,0,30802,28,2048,2500,0.168,0.168,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",...,0.168288,0.027486,0.026430,0.207038,0.226278,0.151056,0.076672,0.054659,1.460316,0.946452
112117,0,30803,42,2048,2500,0.168,0.168,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",...,0.165465,0.026710,0.016028,0.338843,0.551825,0.187382,0.047657,0.027959,0.839741,0.855948
112118,0,30804,29,2048,2500,0.168,0.168,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",...,0.162420,0.112969,0.012846,0.148053,0.434081,0.050531,0.029546,0.078464,0.918192,0.598955


In [147]:
merged_df.dtypes

Follow-up #              int64
Patient ID               int64
Patient Age              int64
OriginalImage[Width      int64
Height]                  int64
                        ...   
feature_2043           float64
feature_2044           float64
feature_2045           float64
feature_2046           float64
feature_2047           float64
Length: 2058, dtype: object

In [148]:
label_classes

['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Effusion',
 'Emphysema',
 'Fibrosis',
 'Hernia',
 'Infiltration',
 'Mass',
 'No Finding',
 'Nodule',
 'Pleural_Thickening',
 'Pneumonia',
 'Pneumothorax']

In [149]:
merged_df.drop(columns=["Patient ID", "Follow-up #"], inplace=True)

In [150]:
merged_df

,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],View Position_PA,Patient Gender_M,Encoded Vectors,feature_0,feature_1,...,feature_2038,feature_2039,feature_2040,feature_2041,feature_2042,feature_2043,feature_2044,feature_2045,feature_2046,feature_2047
0,57,2682,2749,0.143,0.143,1,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.197477,1.071153,...,0.054727,0.000000,0.074869,0.182716,0.414462,0.046612,0.175924,0.550939,0.302225,0.506780
1,58,2894,2729,0.143,0.143,1,1,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.235523,0.750425,...,0.089889,0.017977,0.091583,0.169574,0.507109,0.043761,0.138473,0.652440,0.527583,0.292152
2,58,2500,2048,0.168,0.168,1,1,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.056817,0.692362,...,0.111583,0.033380,0.097974,0.331221,0.068201,0.191576,0.070453,0.076101,0.193749,0.167181
3,80,2500,2048,0.171,0.171,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0.193224,0.793367,...,0.246135,0.008017,0.024985,0.318914,0.556515,0.178968,0.038388,0.102484,0.487945,0.355339
4,74,2500,2048,0.168,0.168,1,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0.123483,0.561670,...,0.079363,0.079900,0.128630,0.254023,0.143975,0.196335,0.038926,0.103639,0.886098,0.308224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,38,2048,2500,0.168,0.168,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]",0.482965,0.764184,...,0.160762,0.039061,0.025498,0.181054,0.096583,0.152477,0.188412,0.016646,0.674780,1.414213
112116,28,2048,2500,0.168,0.168,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0.261510,0.528099,...,0.168288,0.027486,0.026430,0.207038,0.226278,0.151056,0.076672,0.054659,1.460316,0.946452
112117,42,2048,2500,0.168,0.168,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0.158552,0.090915,...,0.165465,0.026710,0.016028,0.338843,0.551825,0.187382,0.047657,0.027959,0.839741,0.855948
112118,29,2048,2500,0.168,0.168,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0.075428,0.199354,...,0.162420,0.112969,0.012846,0.148053,0.434081,0.050531,0.029546,0.078464,0.918192,0.598955


In [151]:
merged_df.to_csv("merged_dataframe.csv", index=False)

In [152]:
from sklearn.model_selection import train_test_split
import numpy as np

# Define Features (X) → All columns **except** the target 'Encoded Vectors'
X = merged_df.drop(columns=['Encoded Vectors'])

# Define Target (y) → Multi-hot encoded vectors
y = np.array(merged_df['Encoded Vectors'].tolist())  # Convert list to numpy array

# Split into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print Shapes to Verify
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")



X_train shape: (89696, 2055), y_train shape: (89696, 15)
X_test shape: (22424, 2055), y_test shape: (22424, 15)


In [153]:
X

,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],View Position_PA,Patient Gender_M,feature_0,feature_1,feature_2,...,feature_2038,feature_2039,feature_2040,feature_2041,feature_2042,feature_2043,feature_2044,feature_2045,feature_2046,feature_2047
0,57,2682,2749,0.143,0.143,1,1,0.197477,1.071153,1.328302,...,0.054727,0.000000,0.074869,0.182716,0.414462,0.046612,0.175924,0.550939,0.302225,0.506780
1,58,2894,2729,0.143,0.143,1,1,0.235523,0.750425,1.329583,...,0.089889,0.017977,0.091583,0.169574,0.507109,0.043761,0.138473,0.652440,0.527583,0.292152
2,58,2500,2048,0.168,0.168,1,1,0.056817,0.692362,1.758177,...,0.111583,0.033380,0.097974,0.331221,0.068201,0.191576,0.070453,0.076101,0.193749,0.167181
3,80,2500,2048,0.171,0.171,1,1,0.193224,0.793367,1.724376,...,0.246135,0.008017,0.024985,0.318914,0.556515,0.178968,0.038388,0.102484,0.487945,0.355339
4,74,2500,2048,0.168,0.168,1,0,0.123483,0.561670,1.289925,...,0.079363,0.079900,0.128630,0.254023,0.143975,0.196335,0.038926,0.103639,0.886098,0.308224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,38,2048,2500,0.168,0.168,1,1,0.482965,0.764184,1.656513,...,0.160762,0.039061,0.025498,0.181054,0.096583,0.152477,0.188412,0.016646,0.674780,1.414213
112116,28,2048,2500,0.168,0.168,1,1,0.261510,0.528099,1.389313,...,0.168288,0.027486,0.026430,0.207038,0.226278,0.151056,0.076672,0.054659,1.460316,0.946452
112117,42,2048,2500,0.168,0.168,1,0,0.158552,0.090915,1.173496,...,0.165465,0.026710,0.016028,0.338843,0.551825,0.187382,0.047657,0.027959,0.839741,0.855948
112118,29,2048,2500,0.168,0.168,1,0,0.075428,0.199354,1.402751,...,0.162420,0.112969,0.012846,0.148053,0.434081,0.050531,0.029546,0.078464,0.918192,0.598955


In [294]:
y

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [155]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


EXPERIMENT #1 : BASELINE MODEL

In [157]:
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Define the model architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer with vectorized data shape
    Dense(32, activation='relu'),  # Hidden layer
    Dense(y_train.shape[1], activation='sigmoid')  # Output layer with 'y_train.shape[1]' units for multi-label classification
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-3),  # Adam optimizer with learning rate
    loss='binary_crossentropy',  # Binary cross-entropy for multi-label classification
    metrics=['accuracy']  # Accuracy metric (you can track precision, recall, etc.)
)

# Model Summary to Verify the Architecture
model.summary()

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 64)             │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,159 (524.06 KB)

 Trainable params: 134,159 (524.06 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.4947 - loss: 0.7404 - val_accuracy: 0.5431 - val_loss: 0.2074
Epoch 2/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 932us/step - accuracy: 0.5337 - loss: 0.2115 - val_accuracy: 0.5426 - val_loss: 0.2037
Epoch 3/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5298 - loss: 0.2081 - val_accuracy: 0.5427 - val_loss: 0.2026
Epoch 4/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 889us/step - accuracy: 0.5259 - loss: 0.2070 - val_accuracy: 0.5422 - val_loss: 0.2021
Epoch 5/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 2s 860us/step - accuracy: 0.5301 - loss: 0.2050 - val_accuracy: 0.5407 - val_loss: 0.2005
Epoch 6/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 2s 859us/step - accuracy: 0.5276 - loss: 0.2052 - val_accuracy: 0.5433 - val_loss: 0.2027
Epoch 7/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 2s 889us/step - accuracy: 0.5295 - loss: 0.2041 - val_accuracy: 0.5191 - val_loss: 0.2013
Epoch 8/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 2s 878us/step - accuracy: 0.5262 - los

In [158]:
# Get the predicted probabilities for the test set
y_pred = model.predict(X_test)

# Apply a threshold (e.g., 0.5) to convert probabilities to binary (0 or 1)
y_pred_bin = (y_pred > 0.5).astype(int)

# Calculate the number of correct predictions (per label and per sample)
correct_predictions = np.sum(y_pred_bin == y_test)

# Calculate the total number of predictions (per sample, per label)
total_predictions = y_test.size

# Calculate the multi-label accuracy
multi_label_accuracy = correct_predictions / total_predictions

print(f"Multi-label Accuracy: {multi_label_accuracy:.4f}")


701/701 ━━━━━━━━━━━━━━━━━━━━ 0s 219us/step
Multi-label Accuracy: 0.9302


In [159]:
y_pred_bin[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [160]:
correct_predictions

312875

In [161]:
total_predictions

336360

EXPERIMENT #2 : Deeper MLP with Dropout

In [163]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
model2 = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='sigmoid')
])

model2.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Optional: Early stopping to avoid overfitting
#early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model2.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
)


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.4659 - loss: 0.8093 - val_accuracy: 0.5429 - val_loss: 0.2174
Epoch 2/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5390 - loss: 0.2154 - val_accuracy: 0.5429 - val_loss: 0.2153
Epoch 3/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5379 - loss: 0.2184 - val_accuracy: 0.5429 - val_loss: 0.2163
Epoch 4/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.5360 - loss: 0.2179 - val_accuracy: 0.5429 - val_loss: 0.2154
Epoch 5/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5387 - loss: 0.2174 - val_accuracy: 0.5429 - val_loss: 0.2154
Epoch 6/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5386 - loss: 0.2180 - val_accuracy: 0.5429 - val_loss: 0.2153
Epoch 7/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5345 - loss: 0.2189 - val_accuracy: 0.5429 - val_loss: 0.2153
Epoch 8/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5383 - loss: 0.2173 - 

In [164]:
# Get the predicted probabilities for the test set
y_pred = model2.predict(X_test)

# Apply a threshold (e.g., 0.5) to convert probabilities to binary (0 or 1)
y_pred_bin = (y_pred > 0.5).astype(int)

# Calculate the number of correct predictions (per label and per sample)
correct_predictions = np.sum(y_pred_bin == y_test)

# Calculate the total number of predictions (per sample, per label)
total_predictions = y_test.size

# Calculate the multi-label accuracy
multi_label_accuracy = correct_predictions / total_predictions

print(f"Multi-label Accuracy: {multi_label_accuracy:.4f}")

701/701 ━━━━━━━━━━━━━━━━━━━━ 0s 394us/step
Multi-label Accuracy: 0.9219


EXPERIMENT #3 :  LeakyReLU Activation

In [166]:
from tensorflow.keras.layers import LeakyReLU

model3 = Sequential([
    Dense(128, input_shape=(X_train.shape[1],)),
    LeakyReLU(alpha=0.1),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dense(y_train.shape[1], activation='sigmoid')
])

model3.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Optional: Early stopping to avoid overfitting
#early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model3.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
)


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.4737 - loss: 0.6124 - val_accuracy: 0.5430 - val_loss: 0.2100
Epoch 2/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5242 - loss: 0.2138 - val_accuracy: 0.4894 - val_loss: 0.2094
Epoch 3/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.5260 - loss: 0.2064 - val_accuracy: 0.5389 - val_loss: 0.1990
Epoch 4/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5239 - loss: 0.2045 - val_accuracy: 0.5022 - val_loss: 0.2042
Epoch 5/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5298 - loss: 0.2016 - val_accuracy: 0.5281 - val_loss: 0.1978
Epoch 6/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5306 - loss: 0.2005 - val_accuracy: 0.5425 - val_loss: 0.1979
Epoch 7/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.5299 - loss: 0.2000 - val_accuracy: 0.5373 - val_loss: 0.1957
Epoch 8/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5275 - loss: 0.1997 - 

In [167]:
# Get the predicted probabilities for the test set
y_pred = model3.predict(X_test)

# Apply a threshold (e.g., 0.5) to convert probabilities to binary (0 or 1)
y_pred_bin = (y_pred > 0.5).astype(int)

# Calculate the number of correct predictions (per label and per sample)
correct_predictions = np.sum(y_pred_bin == y_test)

# Calculate the total number of predictions (per sample, per label)
total_predictions = y_test.size

# Calculate the multi-label accuracy
multi_label_accuracy = correct_predictions / total_predictions

print(f"Multi-label Accuracy: {multi_label_accuracy:.4f}")

701/701 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step
Multi-label Accuracy: 0.9315


EXPERIMENT #4 : TanH + ReLU

In [169]:
model4 = Sequential([
    Dense(128, activation='tanh', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(y_train.shape[1], activation='sigmoid')
])

model4.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Optional: Early stopping to avoid overfitting
#early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model4.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.5232 - loss: 0.2319 - val_accuracy: 0.5429 - val_loss: 0.2161
Epoch 2/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5350 - loss: 0.2197 - val_accuracy: 0.5429 - val_loss: 0.2156
Epoch 3/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5406 - loss: 0.2177 - val_accuracy: 0.5429 - val_loss: 0.2159
Epoch 4/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5342 - loss: 0.2194 - val_accuracy: 0.5429 - val_loss: 0.2154
Epoch 5/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.5386 - loss: 0.2175 - val_accuracy: 0.5429 - val_loss: 0.2158
Epoch 6/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5354 - loss: 0.2186 - val_accuracy: 0.5429 - val_loss: 0.2155
Epoch 7/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5385 - loss: 0.2180 - val_accuracy: 0.5429 - val_loss: 0.2156
Epoch 8/20
2803/2803 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.5360 - loss: 0.2185 - 

In [170]:
# Get the predicted probabilities for the test set
y_pred = model4.predict(X_test)

# Apply a threshold (e.g., 0.5) to convert probabilities to binary (0 or 1)
y_pred_bin = (y_pred > 0.5).astype(int)

# Calculate the number of correct predictions (per label and per sample)
correct_predictions = np.sum(y_pred_bin == y_test)

# Calculate the total number of predictions (per sample, per label)
total_predictions = y_test.size

# Calculate the multi-label accuracy
multi_label_accuracy = correct_predictions / total_predictions

print(f"Multi-label Accuracy: {multi_label_accuracy:.4f}")

701/701 ━━━━━━━━━━━━━━━━━━━━ 0s 269us/step
Multi-label Accuracy: 0.9219


In [215]:
import pickle

# Save the model to a .pickle file
with open('model.pickle', 'wb') as f:
    pickle.dump(model, f)

print("Model has been saved to 'model.pickle'")


Model has been saved to 'model.pickle'


In [218]:
X_test

,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],View Position_PA,Patient Gender_M,feature_0,feature_1,feature_2,...,feature_2038,feature_2039,feature_2040,feature_2041,feature_2042,feature_2043,feature_2044,feature_2045,feature_2046,feature_2047
89645,54,2542,2434,0.143000,0.143000,1,0,0.224810,0.714653,1.293135,...,0.277415,0.074942,0.034360,0.275030,0.228207,0.248295,0.050876,0.051860,1.258434,0.197261
47446,64,2500,2048,0.168000,0.168000,0,1,0.143902,0.241105,1.858710,...,0.043877,0.027666,0.027210,0.167656,0.857629,0.112459,0.001973,0.329413,0.360557,0.666259
57963,6,2992,2033,0.143000,0.143000,1,1,0.728576,0.083379,0.610838,...,0.301594,0.260636,0.028665,0.386106,0.905579,0.080206,0.030516,0.310525,1.652197,0.342029
22863,17,2048,2500,0.171000,0.171000,1,0,0.244323,0.518458,1.417107,...,0.373882,0.036967,0.086907,0.290354,0.155607,0.144260,0.062162,0.156275,1.089894,1.036362
43431,18,2500,2048,0.168000,0.168000,0,1,0.507548,0.613002,1.338565,...,0.200701,0.007847,0.121786,0.049021,0.707941,0.088318,0.025138,0.154303,1.158850,1.311314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86553,63,2992,2991,0.143000,0.143000,1,1,0.064018,0.795458,1.644483,...,0.310028,0.014599,0.052870,0.258561,0.483743,0.111536,0.065362,0.238822,0.744786,0.307270
20458,30,2500,2048,0.168000,0.168000,0,1,0.138182,0.426625,0.811558,...,0.159650,0.082800,0.007841,0.377492,0.203675,0.214312,0.140588,0.067406,0.330233,0.813939
51540,80,2992,2881,0.143000,0.143000,1,1,0.197809,0.631027,1.137532,...,0.292455,0.018509,0.056407,0.314009,0.185001,0.111374,0.173555,0.248931,1.511678,0.127221
84962,34,3056,2544,0.139000,0.139000,0,1,0.125900,0.320098,1.488375,...,0.158599,0.059499,0.002500,0.416880,0.510253,0.057225,0.037763,0.400919,0.325849,0.814512


In [328]:
i = 17795  # or any index
row = X_test.iloc[i].values.reshape(1, -1)
prediction = model.predict(row)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [330]:
prediction

array([[0.15730524, 0.0386307 , 0.07461596, 0.03836269, 0.22694461,
        0.01407254, 0.00845077, 0.00061241, 0.24880658, 0.04993709,
        0.38052142, 0.04452971, 0.02894014, 0.02258314, 0.03969424]],
      dtype=float32)

In [332]:
pred = (prediction > 0.5).astype(int)
pred

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [280]:
y_test[19]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [298]:
import numpy as np

unique_combinations = np.unique(y_test, axis=0)
print(f"Number of unique multi-label combinations in y_test: {len(unique_combinations)}")

# Optional: print a few of them
print("Some unique label combinations:")
print(unique_combinations[:5])  # First 5 unique combinations


Number of unique multi-label combinations in y_test: 408
Some unique label combinations:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]]


In [302]:
y_test.shape

(22424, 15)

In [304]:
import numpy as np

# Define the target label vector you're looking for
target_vector = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

# Compare all rows in y_test to the target_vector
matches = np.all(y_test == target_vector, axis=1)

# Count how many times it appears
count = np.sum(matches)

print(f"Number of samples in y_test with the label {target_vector.tolist()}: {count}")


Number of samples in y_test with the label [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]: 12173


In [310]:
import numpy as np

# Define the target label vector you're looking for
target_vector = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

# Convert the target_vector to a list for comparison
target_list = target_vector.tolist()

# Compare each row in the 'Encoded Vectors' column to the target_list
matches = merged_df['Encoded Vectors'].apply(lambda x: np.array_equal(x, target_list))

# Count how many times the vector appears
count = np.sum(matches)

print(f"Number of samples in 'Encoded Vectors' with the label {target_list}: {count}")



Number of samples in 'Encoded Vectors' with the label [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]: 60361


In [312]:
merged_df.shape

(112120, 2056)

In [326]:
X_train

,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],View Position_PA,Patient Gender_M,feature_0,feature_1,feature_2,...,feature_2038,feature_2039,feature_2040,feature_2041,feature_2042,feature_2043,feature_2044,feature_2045,feature_2046,feature_2047
109402,32,2021,2021,0.194311,0.194311,1,1,0.841712,0.509010,1.087955,...,0.252144,0.000000,0.234208,0.259545,0.561934,0.057299,0.061297,0.155338,1.307174,1.245145
83153,59,2914,2991,0.143000,0.143000,1,1,0.324385,1.057927,1.471220,...,0.169485,0.032224,0.067385,0.229007,0.499014,0.082875,0.097889,0.269410,1.213763,0.677596
12902,48,2048,2500,0.171000,0.171000,1,0,0.140486,0.531314,1.730232,...,0.569280,0.370726,0.044796,0.466035,0.616207,0.177282,0.022216,0.126641,0.613851,0.933715
106281,57,3056,2544,0.139000,0.139000,1,1,0.020203,0.905954,1.857053,...,0.119490,0.004912,0.042406,0.501070,0.363248,0.053934,0.106794,0.081723,0.416990,0.162940
52472,47,2982,2991,0.143000,0.143000,1,1,0.467656,0.446101,1.686128,...,0.205224,0.044609,0.160930,0.421418,0.257745,0.118975,0.152198,0.090817,1.930331,0.966705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76820,71,2544,3056,0.139000,0.139000,1,0,0.191161,0.583617,1.068035,...,0.062853,0.032422,0.119949,0.393288,0.160981,0.042694,0.015287,0.114298,0.404042,0.180844
110268,69,3056,2544,0.139000,0.139000,1,1,0.182750,0.436838,1.274457,...,0.092450,0.001974,0.087935,0.109783,0.252932,0.077969,0.078676,0.065307,0.442601,0.206741
103694,32,2544,3056,0.139000,0.139000,1,1,0.234043,0.769260,1.490279,...,0.403309,0.100086,0.018340,0.485596,0.681211,0.068685,0.060364,0.250472,0.941487,1.002701
860,59,2500,2048,0.168000,0.168000,0,0,0.303974,0.284901,0.966510,...,0.352100,0.000000,0.085013,0.089823,0.559333,0.051933,0.021783,0.307582,1.483242,1.621708
